In [77]:
from pymatgen.core import Structure
from pymatgen.analysis.diffusion.neb.full_path_mapper import MigrationGraph
from pymatgen.analysis.diffusion.neb.io import get_endpoints_from_index

1. Load our supercell as `struct`

In [23]:
struct = Structure.from_file('POSCAR.vasp') # load the structure (POSCAR) 

2. Generate a `MigrationGraph` object and define all Li paths (with `max_distance=4`)

In [74]:
mg = MigrationGraph.with_distance(structure=struct,
                                  migrating_specie="Li",
                                  max_distance=4)

In [75]:
print("Number of unique hops =", len(mg.unique_hops))

Number of unique hops = 3


3. You can also weight the "graph" with hop distance or other parameter such as energy barrier to then get a full path through a material

In [76]:
#mg.assign_cost_to_graph(cost_keys=["hop_distance"])
#paths = [x for x in mg.get_path()]

4. in terms of a vacancy diffusion, we can grab the atom indexes of Li endpoints for our NEB calculations 

In [78]:
unique_hops = mg.unique_hops
paths = {}
for k,v in unique_hops.items():
    paths[k] = {'init':v['iindex'],'final':v['eindex']} # make a dict of init and final atoms so that we can neb

In [79]:
paths

{0: {'init': 0, 'final': 12},
 2: {'init': 0, 'final': 24},
 1: {'init': 0, 'final': 20}}

5. now we can get the structures of each end point (which should be sorted so that no dodgy nebs occur) 

In [80]:
struct = Structure.from_file('POSCAR.vasp') 
endpoints = {}
for path in paths:
    endpoints[path] = get_endpoints_from_index(struct,[paths[path]['init'],paths[path]['final']])

6. Finally write to file

In [81]:
for path in endpoints: #write to files to view later
    endpoints[path][0].to(filename='POSCAR_path_{}_init.vasp'.format(path),fmt='poscar')
    endpoints[path][1].to(filename='POSCAR_path_{}_final.vasp'.format(path),fmt='poscar')

7. No depresso, more espresso